In [1]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Button, Toggle, CheckboxGroup
from bokeh.plotting import figure, show

In [10]:
# Scatterplot: x from uniform(0,1); y from normal(0,1)
# Add 1 particle on click
# Send 100 particles at once
# Reset the screen
# Continuous stream of particles + jde vypnout

from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show

source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(
    title="Add 1 particle on click or send 100 particles at once",
    tools="pan,wheel_zoom,box_zoom,reset",         # (de)activate tools    
    sizing_mode="stretch_width", 
#    max_width=1000, 
    width=500,
    height=500,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,1),
    y_range=(0, 1),
)

p.circle('x', 'y', source=source, size=10, fill_color="red")

# Buttons ----------------------------------------------------------------------
# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")

# Reset the screen
resetbutton = Button(label="Reset", button_type="success")

# Continuous stream of particles
streamtoggle = Button(label="Send continuous stream", button_type="success")
# -----------------------------------------------------------------------------
# JS CALLBACKS

# Def function of normal distribution
jscode_normalPDF = """
    function randn() {
      let u = 0, v = 0;
      
      while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
      while(v === 0) v = Math.random();
      
      let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2.0 * Math.PI * v );
      num = num / 10.0 + 0.5;           // Translate to 0 -> 1
      
      if (num > 1 || num < 0) return randn_bm() // resample between 0 and 1
      return num
    }
"""
    
callback = CustomJS(args=dict(source=source), code=jscode_normalPDF + """
    
    source.data.x.push(randn())
    source.data.y.push(Math.random())

    // update the data source with local changes
    source.change.emit()
    """)

callback2 = CustomJS(args=dict(source=source), code=jscode_normalPDF +"""
    
    let i = 0;
    for (let i = 0; i < 100; i++) {
    source.data.x.push(randn())
    source.data.y.push(Math.random()) 
    }
    
    // update the data source with local changes
    source.change.emit()
    """)

callback_reset = CustomJS(args=dict(source=source), code="""
    
    source.data.x = []
    source.data.y = []
    source.change.emit()
    """)

callback_stream = CustomJS(args=dict(source=source, streamtoggle=streamtoggle), code=jscode_normalPDF + """

    function sendparticle() {
        source.data.x.push(randn());
        source.data.y.push(Math.random());
        source.change.emit();
    }
    
    if (streamtoggle.button_type == "success") {
        const interval = setInterval(sendparticle, 300);
        sendparticle();
        source.interval = interval;
        streamtoggle.button_type = "danger";
        streamtoggle.label = "Stop stream";
        
    } else {
        clearInterval(source.interval);
        streamtoggle.button_type = "success";
        streamtoggle.label = "Send continuous stream";
    }
""")
# -------------------------------------------------------------------------------
# Callbacks to objects

button.js_on_event('button_click', callback)
button100.js_on_event('button_click', callback2)
resetbutton.js_on_event('button_click', callback_reset)
streamtoggle.js_on_event('button_click', callback_stream)

#buttons_layout = column(button, button100, resetbutton)
#show(row(p, buttons_layout))
show(column(p, button, button100, resetbutton, streamtoggle))